In [1]:
#import bigquery
from google.cloud import bigquery
from google.cloud import bigquery_storage
 
#other needed libraries
import os
import pandas as pd
import numpy as np
import pandas_gbq

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/jupyter/.config/gcloud/application_default_credentials.json"

#Instatiate BigQuery Client
client = bigquery.Client()

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [3]:
# Simplified query with the three categories you requested

lac = """
WITH lac_WithAddress_BeforeOrOn AS (
  SELECT
    lac.person_id,
    lac.StartDate,
    lac.EndDate,
    a.SOA AS LSOA_BeforeOrOn,
    ROW_NUMBER() OVER (
      PARTITION BY lac.person_id, lac.StartDate
      ORDER BY DATE(a.DateEvent) DESC
    ) AS rn_beforeoron
  FROM
    `yhcr-prd-bradfor-bia-core.CB_2649.cb_bmbc_ChildrensSocialServices_CiC` lac
  LEFT JOIN
    `yhcr-prd-bradfor-bia-core.CB_2649.tbl_AddressHistory_New` a
      ON lac.person_id = a.person_id
      AND DATE(a.DateEvent) <= lac.StartDate  -- Before OR on (exactly matching original query)
),

lac_WithAddress_After AS (
  SELECT
    lac.person_id,
    lac.StartDate,
    lac.EndDate,
    a.SOA AS LSOA_After,
    ROW_NUMBER() OVER (
      PARTITION BY lac.person_id, lac.StartDate
      ORDER BY DATE(a.DateEvent) ASC
    ) AS rn_after
  FROM
    `yhcr-prd-bradfor-bia-core.CB_2649.cb_bmbc_ChildrensSocialServices_CiC` lac
  LEFT JOIN
    `yhcr-prd-bradfor-bia-core.CB_2649.tbl_AddressHistory_New` a
      ON lac.person_id = a.person_id
      AND DATE(a.DateEvent) > lac.StartDate
),

lac_WithDemographics AS (
  SELECT
    lac.person_id,
    lac.StartDate,
    lac.EndDate,
    lac.EthnicOrigin,
    DATE(p.birth_datetime) AS DateOfBirth,
    p.gender_source_value AS Gender
  FROM
    `yhcr-prd-bradfor-bia-core.CB_2649.cb_bmbc_ChildrensSocialServices_CiC` lac
  LEFT JOIN
    `yhcr-prd-bradfor-bia-core.CB_2649.person` p
      ON lac.person_id = p.person_id
)

SELECT 
  d.person_id,
  d.Gender,      
  d.StartDate,
  d.EndDate,
  d.EthnicOrigin,
  d.DateOfBirth,
  bo.LSOA_BeforeOrOn,
  a.LSOA_After,
  CASE 
    WHEN bo.LSOA_BeforeOrOn IS NOT NULL THEN 'Before/On'
    WHEN bo.LSOA_BeforeOrOn IS NULL AND a.LSOA_After IS NOT NULL THEN 'Only After'
    ELSE 'None'
  END AS LSOA_Availability
FROM
  lac_WithDemographics d
LEFT JOIN
  lac_WithAddress_BeforeOrOn bo
    ON d.person_id = bo.person_id AND d.StartDate = bo.StartDate AND bo.rn_beforeoron = 1
LEFT JOIN
  lac_WithAddress_After a
    ON d.person_id = a.person_id AND d.StartDate = a.StartDate AND a.rn_after = 1
"""

In [4]:
# Simplified query with the three categories you requested

cpp = """
WITH lac_WithAddress_BeforeOrOn AS (
  SELECT
    lac.person_id,
    lac.StartDate,
    lac.EndDate,
    a.SOA AS LSOA_BeforeOrOn,
    ROW_NUMBER() OVER (
      PARTITION BY lac.person_id, lac.StartDate
      ORDER BY DATE(a.DateEvent) DESC
    ) AS rn_beforeoron
  FROM
    `yhcr-prd-bradfor-bia-core.CB_2649.cb_bmbc_ChildrensSocialServices_CPP` lac
  LEFT JOIN
    `yhcr-prd-bradfor-bia-core.CB_2649.tbl_AddressHistory_New` a
      ON lac.person_id = a.person_id
      AND DATE(a.DateEvent) <= lac.StartDate  -- Before OR on (exactly matching original query)
),

lac_WithAddress_After AS (
  SELECT
    lac.person_id,
    lac.StartDate,
    lac.EndDate,
    a.SOA AS LSOA_After,
    ROW_NUMBER() OVER (
      PARTITION BY lac.person_id, lac.StartDate
      ORDER BY DATE(a.DateEvent) ASC
    ) AS rn_after
  FROM
    `yhcr-prd-bradfor-bia-core.CB_2649.cb_bmbc_ChildrensSocialServices_CPP` lac
  LEFT JOIN
    `yhcr-prd-bradfor-bia-core.CB_2649.tbl_AddressHistory_New` a
      ON lac.person_id = a.person_id
      AND DATE(a.DateEvent) > lac.StartDate
),

lac_WithDemographics AS (
  SELECT
    lac.person_id,
    lac.StartDate,
    lac.EndDate,
    lac.EthnicOrigin,
    DATE(p.birth_datetime) AS DateOfBirth,
    p.gender_source_value AS Gender
  FROM
    `yhcr-prd-bradfor-bia-core.CB_2649.cb_bmbc_ChildrensSocialServices_CPP` lac
  LEFT JOIN
    `yhcr-prd-bradfor-bia-core.CB_2649.person` p
      ON lac.person_id = p.person_id
)

SELECT 
  d.person_id,
  d.Gender,      
  d.StartDate,
  d.EndDate,
  d.EthnicOrigin,
  d.DateOfBirth,
  bo.LSOA_BeforeOrOn,
  a.LSOA_After,
  CASE 
    WHEN bo.LSOA_BeforeOrOn IS NOT NULL THEN 'Before/On'
    WHEN bo.LSOA_BeforeOrOn IS NULL AND a.LSOA_After IS NOT NULL THEN 'Only After'
    ELSE 'None'
  END AS LSOA_Availability
FROM
  lac_WithDemographics d
LEFT JOIN
  lac_WithAddress_BeforeOrOn bo
    ON d.person_id = bo.person_id AND d.StartDate = bo.StartDate AND bo.rn_beforeoron = 1
LEFT JOIN
  lac_WithAddress_After a
    ON d.person_id = a.person_id AND d.StartDate = a.StartDate AND a.rn_after = 1
"""

In [5]:
# Simplified query with the three categories you requested

cinp = """
WITH lac_WithAddress_BeforeOrOn AS (
  SELECT
    lac.person_id,
    lac.StartDate,
    lac.EndDate,
    a.SOA AS LSOA_BeforeOrOn,
    ROW_NUMBER() OVER (
      PARTITION BY lac.person_id, lac.StartDate
      ORDER BY DATE(a.DateEvent) DESC
    ) AS rn_beforeoron
  FROM
    `yhcr-prd-bradfor-bia-core.CB_2649.cb_bmbc_ChildrensSocialServices_CiNP` lac
  LEFT JOIN
    `yhcr-prd-bradfor-bia-core.CB_2649.tbl_AddressHistory_New` a
      ON lac.person_id = a.person_id
      AND DATE(a.DateEvent) <= lac.StartDate  -- Before OR on (exactly matching original query)
),

lac_WithAddress_After AS (
  SELECT
    lac.person_id,
    lac.StartDate,
    lac.EndDate,
    a.SOA AS LSOA_After,
    ROW_NUMBER() OVER (
      PARTITION BY lac.person_id, lac.StartDate
      ORDER BY DATE(a.DateEvent) ASC
    ) AS rn_after
  FROM
    `yhcr-prd-bradfor-bia-core.CB_2649.cb_bmbc_ChildrensSocialServices_CiNP` lac
  LEFT JOIN
    `yhcr-prd-bradfor-bia-core.CB_2649.tbl_AddressHistory_New` a
      ON lac.person_id = a.person_id
      AND DATE(a.DateEvent) > lac.StartDate
),

lac_WithDemographics AS (
  SELECT
    lac.person_id,
    lac.StartDate,
    lac.EndDate,
    lac.EthnicOrigin,
    DATE(p.birth_datetime) AS DateOfBirth,
    p.gender_source_value AS Gender
  FROM
    `yhcr-prd-bradfor-bia-core.CB_2649.cb_bmbc_ChildrensSocialServices_CiNP` lac
  LEFT JOIN
    `yhcr-prd-bradfor-bia-core.CB_2649.person` p
      ON lac.person_id = p.person_id
)

SELECT 
  d.person_id,
  d.Gender,      
  d.StartDate,
  d.EndDate,
  d.EthnicOrigin,
  d.DateOfBirth,
  bo.LSOA_BeforeOrOn,
  a.LSOA_After,
  CASE 
    WHEN bo.LSOA_BeforeOrOn IS NOT NULL THEN 'Before/On'
    WHEN bo.LSOA_BeforeOrOn IS NULL AND a.LSOA_After IS NOT NULL THEN 'Only After'
    ELSE 'None'
  END AS LSOA_Availability
FROM
  lac_WithDemographics d
LEFT JOIN
  lac_WithAddress_BeforeOrOn bo
    ON d.person_id = bo.person_id AND d.StartDate = bo.StartDate AND bo.rn_beforeoron = 1
LEFT JOIN
  lac_WithAddress_After a
    ON d.person_id = a.person_id AND d.StartDate = a.StartDate AND a.rn_after = 1
"""

In [6]:
cpp = pandas_gbq.read_gbq(cpp)
lac = pandas_gbq.read_gbq(lac)
cinp = pandas_gbq.read_gbq(cinp)

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|


In [12]:
cpp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4112 entries, 0 to 4111
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   person_id          4112 non-null   object
 1   Gender             4109 non-null   object
 2   StartDate          4112 non-null   dbdate
 3   EndDate            3216 non-null   dbdate
 4   EthnicOrigin       4112 non-null   object
 5   DateOfBirth        4110 non-null   dbdate
 6   LSOA_BeforeOrOn    2731 non-null   object
 7   LSOA_After         2420 non-null   object
 8   LSOA_Availability  4112 non-null   object
dtypes: dbdate(3), object(6)
memory usage: 289.2+ KB


In [7]:
cpp.LSOA_Availability.value_counts()

LSOA_Availability
Before/On     2731
None          1102
Only After     279
Name: count, dtype: int64

In [8]:
lac.LSOA_Availability.value_counts()

LSOA_Availability
Before/On     1336
None           471
Only After     113
Name: count, dtype: int64

In [9]:
cinp.LSOA_Availability.value_counts()

LSOA_Availability
Before/On     3884
None          1631
Only After     234
Name: count, dtype: int64

In [13]:
cinp.loc[cinp['LSOA_Availability'] == 'Only After']

,person_id,Gender,StartDate,EndDate,EthnicOrigin,DateOfBirth,LSOA_BeforeOrOn,LSOA_After,LSOA_Availability
0,07001BFAE5B1F9A4EFD2030C35F18102940DCB982A80DD...,M,2019-10-07,2019-12-09,White - British,2019-12-15,None,E01011518,Only After
2,EAABC389243D6A64B7DCB3E5B29ACE2A529185D96CD778...,M,2021-03-05,NaT,Information Not Yet Obtained,2021-05-15,None,E01010699,Only After
3,9ECE7CD04B29A2EAD2AABCD02DDDFBD8352ADD7729F17A...,F,2019-10-11,2019-12-02,White - British,2019-10-15,None,E01027632,Only After
4,D9CF1066D5C6AF087C9988C8BA4DC8505D28E8B4E2D909...,M,2019-10-16,2020-02-13,White - British,2020-03-15,None,E01012664,Only After
6,1CC402E201DBDE8C654F1BB8CCEDD48A4FF33741536959...,M,2021-03-16,NaT,Information Not Yet Obtained,2021-05-15,None,,Only After
...,...,...,...,...,...,...,...,...,...
5717,5589E9C8E3E5C1547E41B71C683E90C9A91A11923E2942...,M,2020-06-22,2020-10-28,Asian/British Asian - Pakistani,2020-09-15,None,E01010736,Only After
5727,973EFEF21C0D4DC2466A1968243977AC13198FD5BF9290...,F,2020-10-22,2020-11-18,Information Not Yet Obtained,2006-08-15,None,E01024900,Only After
5732,ABBF1EB257144AF19B1D921C1AB6507D485C685C6FEB32...,F,2020-12-08,2021-03-31,Asian/British Asian - Pakistani,2009-03-15,None,E01010833,Only After
5738,2437809A16594000C836E493947E567029228F4864EEF4...,M,2019-10-02,2020-04-01,Asian/British Asian - Other,2019-11-15,None,E01010610,Only After
